In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from keras import utils
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, Flatten, Dropout, SimpleRNN 
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [ ]:
# Максимальное количество слов 
num_words = 10000
# Максимальная длина обращения
max_len = 9

In [ ]:
#Загрузка сета
train = pd.read_table('data/train.tsv', encoding='latin-1', sep='\t', header=None, names=['phrase', 'history', 'binary'])

In [ ]:
# Токенезируем текст, делаем словарь
a = train['phrase']
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(a)
sequences = tokenizer.texts_to_sequences(a)

In [ ]:
print(tokenizer.word_index)

In [ ]:
y_train = np_utils.to_categorical(train['binary'])
x_train = pad_sequences(sequences, maxlen=max_len)

In [ ]:
model_cnn = Sequential()
model_cnn.add(Embedding(num_words, 32, input_length=max_len))
model_cnn.add(Conv1D(250, 5, padding='valid', activation='relu'))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dense(1, activation='sigmoid'))

In [ ]:
model_cnn.compile(optimizer='Adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])


In [ ]:
model_cnn_save_path = 'best_model_cnn.h5'
checkpoint_callback_lstm = ModelCheckpoint(model_cnn_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)     

In [ ]:
history = model.fit(x_train, 
                    y_train, 
                    epochs=10,
                    batch_size=12,
                    validation_split=0.10,
                    callbacks=[checkpoint_callback_lstm])

In [ ]:
plt.plot(history.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()